In [1]:
#Install MNE package
!pip install mne -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 36.5 MB/s eta 0:00:00


Dataset - Epilepsy data collected using EMOTIVE with 128Hz sampling frequency in the Ebonyi State, Nigeria. There are a total of 112 subjects

In [2]:
!wget https://zenodo.org/records/1252141/files/EEGs_Nigeria.zip

--2024-07-10 07:43:53--  https://zenodo.org/records/1252141/files/EEGs_Nigeria.zip
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276402063 (264M) [application/octet-stream]
Saving to: ‘EEGs_Nigeria.zip’

EEGs_Nigeria.zip    100%[===================>] 263.60M  13.5MB/s    in 21s     

2024-07-10 07:44:15 (12.7 MB/s) - ‘EEGs_Nigeria.zip’ saved [276402063/276402063]



In [3]:
#Unzipping the files
from zipfile import ZipFile
data = ZipFile('EEGs_Nigeria.zip')
data.extractall()

In [4]:
#Libraries to read the data
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [5]:
#Reading the data meta-data
meta_df= pd.read_csv('https://zenodo.org/records/1252141/files/metadata_nigeria.csv')
meta_df.head()

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz


In [6]:
#Separating Epilepsy Vs Control Subjects
EP_Sub= meta_df['subject.id'][meta_df['Group']=='epilepsy']
CT_Sub= meta_df['subject.id'][meta_df['Group']=='control']

In [7]:
#Reading the data using the subject information
#Epilepsy=[pd.read_csv('EEGs_Nigeria/signal-{}-1.csv.gz'.format(i), compression='gzip') for i in EP_Sub]
#Control=[pd.read_csv('EEGs_Nigeria/signal-{}-2.csv.gz'.format(i), compression='gzip') for i in CT_Sub]
def read_files(prefix, subject_list):
    data = []
    for i in subject_list:
        for suffix in ['1', '2']:
            try:
                file_path = f'EEGs_Nigeria/signal-{i}-{suffix}.csv.gz'
                data.append(pd.read_csv(file_path, compression='gzip'))
            except FileNotFoundError:
                pass
    return data

# Reading the data using the subject information
Epilepsy = read_files('EEGs_Nigeria/signal', EP_Sub)
Control = read_files('EEGs_Nigeria/signal', CT_Sub)

In [8]:
Epilepsy[0].head() #Display the content of the epilepsy list

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4036,330.221053,4134,-213.473684,155.242105,3712,322.421053,3935,4169,...,4,4,4,4,4,4,4,4,4,4
1,2,4053,341.505263,4138,-211.936842,160.368421,3715,321.905263,3944,4173,...,4,4,4,4,4,4,4,4,4,4
2,3,4053,-344.757895,4137,-212.957895,156.778947,3713,322.421053,3939,4174,...,4,4,4,4,4,4,4,4,4,4
3,4,4044,343.557895,4136,-212.452632,147.042105,3713,323.452632,3930,4170,...,4,4,4,4,4,4,4,4,4,4
4,5,4049,-344.242105,4131,-213.989474,141.400000,3712,319.347368,3927,4170,...,4,4,4,4,4,4,4,4,4,4


In [9]:
#Removing unneeded channels
Epilepsy=[i.iloc[:,1:15] for i in Epilepsy]
Control=[i.iloc[:,1:15] for i in Control]

In [10]:
#Converting csv data to MNE
import mne
def convert_csv2mne(sub):
  info= mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
  info.set_montage('standard_1020')
  data=mne.io.RawArray(sub.T, info)
  data.set_eeg_reference()
  data.filter(l_freq=0.1,h_freq=45)
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
  epochs=epochs.drop_bad()
  return epochs

In [11]:
#Convert all the data files (frames) to mne object
%%capture
Epilepsy = [convert_csv2mne(i) for i in Epilepsy]
Control = [convert_csv2mne(i) for i in Control]

In [12]:
%%capture
#Concatenate epochs because they are still in a list
Epilepsy_epochs = mne.concatenate_epochs(Epilepsy)
Control_epochs = mne.concatenate_epochs(Control)

Creating Labels and groups

In [13]:
#Craeting groups and labels for the dataset
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [14]:
Epilepsy_group.shape, Control_group.shape, Epilepsy_label.shape, Control_label.shape

((8037,), (6261,), (8037,), (6261,))

In [15]:
#Combining the data
data=mne.concatenate_epochs([Epilepsy_epochs, Control_epochs]) #Mne object
group=np.concatenate([Epilepsy_group, Control_group]) #Numpy array
label=np.concatenate([Epilepsy_label, Control_label]) #Numpy array
print(len(data), len(group), len(label))

Not setting metadata
14298 matching events found
No baseline correction applied
14298 14298 14298


**Feature Extraction - Power Spectral Density**

**Power Spectral Density (PSD)** is a measure that describes how the power of a signal is distributed over different frequency components. In the context of EEG signal feature extraction, PSD is used to analyze the frequency content of brain signals, which is crucial for understanding brain function and diagnosing neurological conditions.

**Importance in EEG:**
</br>**Feature Extraction:** PSD helps in identifying dominant frequency bands (delta, theta, alpha, beta, gamma) in EEG signals. These bands are associated with different brain states and cognitive processes.
</br>**Noise Reduction:** By examining the PSD, it's possible to filter out noise and artifacts from the EEG data, enhancing the signal quality for further analysis.
</br>**Clinical Applications:** PSD is used in diagnosing and monitoring conditions like epilepsy, sleep disorders, and brain injuries.

**Methods of computing PSD**
1. **Fast Fourier Transform (FFT)** is an algorithm that computes the Discrete Fourier Transform (DFT) of a signal, which transforms a time-domain signal into its frequency-domain representation. While FFT is efficient, it can produce noisy estimates of the PSD, especially for non-stationary signals like EEG, and might require windowing to reduce spectral leakage.

2. **Welch's method** is an improved technique for estimating the PSD that reduces the variance of the estimate compared to a single FFT. The signal is divided into overlapping segments. Each segment is windowed (commonly with a Hamming window) to reduce edge effects. FFT is computed for each windowed segment. The squared magnitudes of the FFTs of the segments are averaged to produce the final PSD estimate.

In [16]:
%%capture
!pip install --upgrade mne

In [17]:
#source: https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5 * n_channels]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
    }

    spectrum = epochs.compute_psd(picks="eeg", fmin=0.5, fmax=30.0)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band)

    return np.concatenate(X, axis=1)

In [18]:
#For Cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [22]:
#Loop through the epochs, and get the mean PSDs for each bands of each channel (features)
%%capture
features = []
for d in range(len(data)): #get the features from each epoch and append into a list
  features.append(eeg_power_band(data[d]))

In [23]:
#finally we convert the list from above into a numpy array
features = np.concatenate(features)
features.shape

(14298, 70)

In [24]:
#evaluating the performance of the RandomForestClassifier (basically how well our model classified things) using 5-fold cross validation
clf = RandomForestClassifier()
accuracies = cross_val_score(clf, features, label, groups = group, cv = 5)
print("5-fold accuracies: ", accuracies)
print("Average accuracy: ", np.mean(accuracies))

5-fold accuracies:  [0.60979021 0.66958042 0.68496503 0.64672963 0.62049668]
Average accuracy:  0.6463123934477555


In [25]:
# Train the final model on the entire dataset
clf.fit(features, label)

RandomForestClassifier()

In [27]:
# Import the joblib library
import joblib

# Save the trained model
joblib.dump(clf, 'eeg_classifier.pkl')

['eeg_classifier.pkl']